# Strategy design pattern

The strategy design pattern allows the selection of an algorithm using a common interface. First, we will see how to implement it using classes and afterwards, we will simplify it by using first order functions.

## Strategy using classes

In [6]:
from abc import ABC, abstractclassmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional

class Customer(NamedTuple):
    name: str
    fidelity: int

class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal: # Question: we do not type annotate self?
        return self.price * self.quantity

class Order(NamedTuple): # The context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0)) # Why the start param?

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self) -> str:
        return f'Order total: {self.total():.2f} due: {self.due():.2f}'

class Promotion(ABC): # the Strategy: an abstract base class
    @abstractclassmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""
    

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)
        

class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount

class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""


    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)


In [8]:
# Test of class defined above

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = (LineItem('banana', 4, Decimal('0.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))
    
Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())

banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('banana', 30, Decimal('.5')))

Order(joe, banana_cart, BulkItemPromo())
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))
Order(joe, long_cart, LargeOrderPromo())

Order total: 10.00 due: 9.30